# Scrittopoli - Giornata 3

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [2]:
# Generale

day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day


calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])



# Giornata 1

label="giornata_1"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"

schedina_g1=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g1=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g1=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g1=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")

# Giornata 2

label="giornata_2"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"

schedina_g2=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g2=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g2=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g2=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")

GIORNATA=3

# Squadre

squadre_csv="data/squadre3.csv" 
squadre=pd.read_csv(squadre_csv,sep=":",quotechar='"',header=0)
squadre=squadre.set_index(["squadra"])


## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
quadro=calendario.loc[GIORNATA].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano




,,squadra 1
girone,partita,
A,2,D Euforiche


In [4]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs                      squadra 2
girone partita                                                           
A      1              La threesome  contro                     Gli ultimi
B      1               Green House  contro                 the 3 BrOthers
       2                I Bananari  contro  Gli scrittori del quartierino
C      1             Living Stones  contro             Armata Brancaleone
       2        Retroguardia Trash  contro                    Le TreMende

In [5]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>La threesome</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli ultimi</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Armata Brancaleone</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style

## Fase 1. Dalle formazioni ai match

In [6]:
#squadre=scrittopoli.copy()
#squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_formazioni(GIORNATA)
s.set_formazioni(GIORNATA,riposano["squadra 1"].values,squadre.index.unique(),old_data)


#formazioni=s.get_formazioni(GIORNATA)
#formazioni



In [7]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
#data=[]
#for sq in squadre.index.unique():
#    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
#    data.append([sq,capitano,riserva]+titolari)
#formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
#formazioni=formazioni.set_index("squadra")
#formazioni["riposa"]=""
#formazioni.set_value(riposano.iloc[0]["squadra 1"],"riposa","(riposa)")

formazioni=s.get_formazioni(GIORNATA)


In [8]:
formazioni

,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
Green House,,Bango Skank,Joyopi,Bango Skank,Plata
La threesome,,mina99,Komorebi,mina99,Luna
I Bananari,,simone volponi,Niko,simone volponi,massimopud
Armata Brancaleone,,Ella F.,Skorpiòs,Ella F.,AdStr
Living Stones,,Macleo,Vincenzo Iennaco,Macleo,Ljuset
Retroguardia Trash,,Andrea28,Andrea28,Eudes,AlexComan
Le TreMende,,Emy,Emy,sefora,Thea
Gli scrittori del quartierino,,Marty12,Marty12,camparino,Federico72
the 3 BrOthers,,AryaSophia,AryaSophia,ettore70,Patrick D'anello


In [9]:
a=formazioni[["riserva"]].copy()
b=formazioni[["titolare 1"]].copy()
c=formazioni[["titolare 2"]].copy()
a.columns=["giocatore"]
b.columns=["giocatore"]
c.columns=["giocatore"]
giocatori=a.append(b).append(c)
giocatori=giocatori[giocatori["giocatore"]!=""]
giocatori=giocatori.reset_index().set_index("giocatore")
giocatori

,squadra
giocatore,
Joyopi,Green House
Komorebi,La threesome
Niko,I Bananari
Skorpiòs,Armata Brancaleone
Vincenzo Iennaco,Living Stones
Andrea28,Retroguardia Trash
Emy,Le TreMende
Marty12,Gli scrittori del quartierino
AryaSophia,the 3 BrOthers


In [10]:
importlib.reload(melo)

elo_g2=elo_g_g2.copy()
elo_g2.columns=["squadra","rank %d" % (GIORNATA-1)]

elo_g2.merge(giocatori,left_index=True,right_index=True,how="outer")

giocatori=giocatori.reset_index().set_index(["squadra","giocatore"])
elo_g2=elo_g2.reset_index().set_index(["squadra","giocatore"])
elo_g2=elo_g2.merge(giocatori,left_index=True,right_index=True,how="outer")
elo_g2=elo_g2.fillna(1500)
elo_g2=elo_g2.reset_index().set_index("giocatore")

# c'è una componente casuale, una volta pubblicati vanno fissati
#accoppiamenti=melo.calcola_accoppiamenti(GIORNATA,formazioni,calendario,elo_g2)
#accoppiamenti.reset_index().as_matrix()

#data=...
data=[['A', 1, 'La threesome', 'mina99', 'Komorebi', 'Luna', 'mina99'],
       ['A', 1, 'Gli ultimi', 'crazycat', 'Steto', 'crazycat', 'caipiroska'],
       ['B', 1, 'Green House', 'Bango Skank', 'Joyopi', 'Bango Skank',
        'Plata'],
       ['B', 1, 'the 3 BrOthers', 'AryaSophia', 'AryaSophia',
        "Patrick D'anello", 'ettore70'],
       ['B', 2, 'I Bananari', 'simone volponi', 'Niko', 'massimopud',
        'simone volponi'],
       ['B', 2, 'Gli scrittori del quartierino', 'Marty12', 'Marty12',
        'camparino', 'Federico72'],
       ['C', 1, 'Living Stones', 'Macleo', 'Vincenzo Iennaco', 'Ljuset',
        'Macleo'],
       ['C', 1, 'Armata Brancaleone', 'Ella F.', 'Skorpiòs', 'Ella F.',
        'AdStr'],
       ['C', 2, 'Retroguardia Trash', 'Andrea28', 'Andrea28', 'Eudes',
        'AlexComan'],
       ['C', 2, 'Le TreMende', 'Emy', 'Emy', 'sefora', 'Thea']]
accoppiamenti=pd.DataFrame(data,columns=["girone","partita","squadra","capitano",
                                         "riserva","match 1",
                                         "match 2"]).set_index(["girone","partita"])


accoppiamenti





squadra        capitano  \
girone partita                                                  
A      1                         La threesome          mina99   
       1                           Gli ultimi        crazycat   
B      1                          Green House     Bango Skank   
       1                       the 3 BrOthers      AryaSophia   
       2                           I Bananari  simone volponi   
       2        Gli scrittori del quartierino         Marty12   
C      1                        Living Stones          Macleo   
       1                   Armata Brancaleone         Ella F.   
       2                   Retroguardia Trash        Andrea28   
       2                          Le TreMende             Emy   

                         riserva           match 1         match 2  
girone partita                                                      
A      1                Komorebi              Luna          mina99  
       1                   Steto          crazycat      caipiroska  
B      1                  Joyopi       Bango Skank           Plata  
       1              AryaSophia  Patrick D'anello        ettore70  
       2                    Niko        massimopud  simone volponi  
       2                 Marty12         camparino      Federico72  
C      1        Vincenzo Iennaco            Ljuset          Macleo  
       1                Skorpiòs           Ella F.           AdStr  
       2                Andrea28             Eudes       AlexComan  
       2                     Emy            sefora            Thea

In [11]:
# Codice html per il post di riepilogo sugli accoppiamenti

prev_girone=""
for girone,partita in accoppiamenti.index.unique():
    if prev_girone!=girone:
        print("<p>&nbsp;</p>")
        print("<p><strong>Girone %s</strong></p>" % girone)
        prev_girone=girone
    p1=accoppiamenti.loc[girone,partita].iloc[0]
    p2=accoppiamenti.loc[girone,partita].iloc[1]
    print("<p>%d. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>&nbsp;</p>
<p><strong>Girone A</strong></p>
<p>1. La threesome - Gli ultimi</p>
<p style='margin-left: 40px;'>match 1: Luna - crazycat</p>
<p style='margin-left: 40px;'>match 2: mina99 - caipiroska</p>
<p>&nbsp;</p>
<p><strong>Girone B</strong></p>
<p>1. Green House - the 3 BrOthers</p>
<p style='margin-left: 40px;'>match 1: Bango Skank - Patrick D'anello</p>
<p style='margin-left: 40px;'>match 2: Plata - ettore70</p>
<p>2. I Bananari - Gli scrittori del quartierino</p>
<p style='margin-left: 40px;'>match 1: massimopud - camparino</p>
<p style='margin-left: 40px;'>match 2: simone volponi - Federico72</p>
<p>&nbsp;</p>
<p><strong>Girone C</strong></p>
<p>1. Living Stones - Armata Brancaleone</p>
<p style='margin-left: 40px;'>match 1: Ljuset - Ella F.</p>
<p style='margin-left: 40px;'>match 2: Macleo - AdStr</p>
<p>2. Retroguardia Trash - Le TreMende</p>
<p style='margin-left: 40px;'>match 1: Eudes - sefora</p>
<p style='margin-left: 40px;'>match 2: AlexComan - Thea</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [12]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

# togliere quando c'è la versione definitiva delle formazioni
old_data=s.get_giornata(GIORNATA)
#old_data=None
s.set_giornata(GIORNATA,accoppiamenti,gironi,old_data)
#old_data

### Tabella ammissioni

In [13]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
ammissioni=s.get_giornata(GIORNATA)
ammissioni=ammissioni[["base","verifiche"]]

old_cols=ammissioni.columns.tolist()
new_cols=[c[1] for c in old_cols]
new_cols

ammissioni.columns=new_cols
a=accoppiamenti.reset_index().set_index("match 1")[["girone","partita","squadra","capitano","riserva"]]
a["match"]=1
a
b=accoppiamenti.reset_index().set_index("match 2")[["girone","partita","squadra","capitano","riserva"]]
b["match"]=2

ammissioni=ammissioni.merge(a.append(b),left_index=True,right_index=True)
new_index=["girone","partita","match","squadra"]
new_columns=["index","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]

ammissioni=ammissioni.reset_index().set_index(new_index).sort_index()[new_columns]
new_columns=["titolare","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]
ammissioni.columns=new_columns

In [14]:
def calc_ammissioni(row):
    row["sostituzione"]=""
    if row["ingresso riserva"]:
        sost="("+row["riserva"]+" da "+row["ingresso riserva"]+")"
        row["sostituzione"]=sost
    row["punti penalità"]=""
    if int(row["penalità"])==0: return row
    if abs(int(row["penalità"]))==1:
        row["punti penalità"]="1 punto di penalità"
    else:
        row["punti penalità"]="%d punti di penalità" % abs(int(row["penalità"]))
    return row

ammissioni=ammissioni.apply(calc_ammissioni,axis=1)
ammissioni

titolare  \
girone partita match squadra                                           
A      1       1     Gli ultimi                             crazycat   
                     La threesome                               Luna   
               2     Gli ultimi                           caipiroska   
                     La threesome                             mina99   
B      1       1     Green House                         Bango Skank   
                     the 3 BrOthers                 Patrick D'anello   
               2     Green House                               Plata   
                     the 3 BrOthers                         ettore70   
       2       1     Gli scrittori del quartierino         camparino   
                     I Bananari                           massimopud   
               2     Gli scrittori del quartierino        Federico72   
                     I Bananari                       simone volponi   
C      1       1     Armata Brancaleone                      Ella F.   
                     Living Stones                            Ljuset   
               2     Armata Brancaleone                        AdStr   
                     Living Stones                            Macleo   
       2       1     Le TreMende                              sefora   
                     Retroguardia Trash                        Eudes   
               2     Le TreMende                                Thea   
                     Retroguardia Trash                    AlexComan   

                                                             riserva  \
girone partita match squadra                                           
A      1       1     Gli ultimi                                Steto   
                     La threesome                           Komorebi   
               2     Gli ultimi                                Steto   
                     La threesome                           Komorebi   
B      1       1     Green House                              Joyopi   
                     the 3 BrOthers                       AryaSophia   
               2     Green House                              Joyopi   
                     the 3 BrOthers                       AryaSophia   
       2       1     Gli scrittori del quartierino           Marty12   
                     I Bananari                                 Niko   
               2     Gli scrittori del quartierino           Marty12   
                     I Bananari                                 Niko   
C      1       1     Armata Brancaleone                     Skorpiòs   
                     Living Stones                  Vincenzo Iennaco   
               2     Armata Brancaleone                     Skorpiòs   
                     Living Stones                  Vincenzo Iennaco   
       2       1     Le TreMende                                 Emy   
                     Retroguardia Trash                     Andrea28   
               2     Le TreMende                                 Emy   
                     Retroguardia Trash                     Andrea28   

                                                   ingresso riserva  \
girone partita match squadra                                          
A      1       1     Gli ultimi                                       
                     La threesome                                     
               2     Gli ultimi                                       
                     La threesome                                     
B      1       1     Green House                                      
                     the 3 BrOthers                                   
               2     Green House                                      
                     the 3 BrOthers                                   
       2       1     Gli scrittori del quartierino                    
                     I Bananari                                       
               2     Gli scrittori del

In [15]:

#old_cols=ammissioni.columns.tolist()
#new_cols=[c[1] for c in old_cols]
#new_cols

#ammissioni.columns=new_cols

def str_penalita(p):
    p=int(p)
    if p==0: return ""
    return "%d goal" % p
    

ammissioni["giocatore"]=ammissioni["titolare"]+" "+ammissioni["sostituzione"]
ammissioni["str penalità"]=ammissioni["penalità"].apply(str_penalita)

#ammissioni[["giocatore","titolo","lunghezza","str penalità"]]

mu.to_table_html_multi(ammissioni[["giocatore","titolo",
                                   "lunghezza","str penalità"]])

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>match</th>
<th>squadra</th>
<th>giocatore</th>
<th>titolo</th>
<th>lunghezza</th>
<th>str penalità</th>
</tr>
<tr>
<th rowspan='4'>A</th>
<th rowspan='8'>1</th>
<th rowspan='2'>1</th>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>crazycat </td>
<td style='text-align:center;'>La ruota panoramica in un sottopassaggio</td>
<td style='text-align:center;'>6953</td>
<td style='text-align:center;'></td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>Luna </td>
<td style='text-align:center;'>Per una tezza di tè</td>
<td style='text-align:center;'>7981</td>
<td style='text-align:center;'>-1 goal</td>
</tr>
<tr>
<th rowspan='2'>2</th>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>caipiroska </td>
<td style='text-align:center;'>Semino d'ortica</td>
<td style='text-align:center;'>7757</td>
<td style='text-align:center;'></td>
</tr>
<tr>
<th rowspan

## Fase 3. Recupero dati

In [43]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(GIORNATA)


In [44]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
importlib.reload(mu)

SIMULA=False
if SIMULA:
    # solo prima delle ammissioni
    #voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
    #voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
    #voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
    #voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
    #voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
    #voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
    #voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)

    for G,Gpart_list,altri in [ ("A",[1],[("B",1),("B",2),("C",1),("C",2)]),
                               ("B",[1,2],[("A",1),("C",1),("C",2)]),
                               ("C",[1,2],[("A",1),("B",1),("B",2)]) ]:
        for sq in gironi[gironi[G]!="(riposo)"][G]:
            for part in Gpart_list:
                for m in ["match 1","match 2"]:
                    giocatore_1=accoppiamenti.loc[G,part][m].iloc[0]
                    giocatore_2=accoppiamenti.loc[G,part][m].iloc[1]
                    goal_1=0
                    goal_2=0
                    voti.set_value(giocatore_1,(G,sq),goal_1)
                    voti.set_value(giocatore_2,(G,sq),goal_2)
            
            for gir,part in altri:
                for m in ["match 1","match 2"]:
                    giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
                    giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
                    goal_1=random.randint(0,2)
                    goal_2=2-goal_1
                    voti.set_value(giocatore_1,(G,sq),goal_1)
                    voti.set_value(giocatore_2,(G,sq),goal_2)
    # solo prima delle ammissioni
    #voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
    #for k in ["tag","link","link commento","commento valido"]:
    #    voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)


In [45]:
voti["verifiche","penalità"]=pd.to_numeric(voti["verifiche","penalità"],downcast="signed")
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti["base","goal giocatore"]=pd.to_numeric(voti["base","goal giocatore"],downcast="signed")
voti

base  \
                 ingresso riserva   
titolare                            
Luna                                
mina99                              
crazycat                            
caipiroska                          
Bango Skank                         
Plata                               
Patrick D'anello                    
ettore70                            
massimopud                          
simone volponi                      
camparino                           
Federico72                          
Ljuset                              
Macleo                              
Ella F.               30/12 18:55   
AdStr                               
Eudes                               
AlexComan                           
sefora                              
Thea                                

                                                                    verifiche  \
                                                             titolo lunghezza   
titolare                                                                        
Luna                                            Per una tezza di tè      7981   
mina99            Non è forse vero che l'attesa della tempesta è...      7985   
crazycat                   La ruota panoramica in un sottopassaggio      6953   
caipiroska                                          Semino d'ortica      7757   
Bango Skank                                    Il vecchio guardiano      7953   
Plata                                Un ennesimo racconto di Natale      7121   
Patrick D'anello                                      Ultimi attimi      3546   
ettore70                                             Amari confetti      7846   
massimopud                          Verdi colline dell'alto Zambesi      7756   
simone volponi                  Ora raccogli tempesta... raccoglila      7966   
camparino                               Avventurieri nella tempesta      4997   
Federico72                                               Il relitto      7936   
Ljuset                                                     Tempesta      7569   
Macleo                                           Il quinto naufrago      3489   
Ella F.                          Ichibi Lokuphila (lago della vita)      6837   
AdStr                                                 Le Vele verdi      7869   
Eudes                                Personaggi di un autore minore      7082   
AlexComan                                        Tempesta a fumetti      7999   
sefora                                             Storia dell'arte      7932   
Thea                                                       Il suono      7653   

                                                                           A  \
                 tag link link commento commento valido penalità D Euforiche   
titolare                                                                       
Luna               x                  x               x       -1           0   
mina99             x    x             x               x        0           0   
crazycat           x    x             x               x        0           0   
caipiroska         x    x             x               x        0           0   
Bango Skank        x    x             x               x        0           2   
Plata              x    x             x               x        0           2   
Patrick D'anello   x    x             x               x        0           0   
ettore70           x    x             x               x        0           0   
massimopud         x    x             x               x        0           0   
simone volponi     x    x             x               x        0           1   
camparino          x    x             x               x        0           2   
Federico72         x    x             x               x        0           1   
Ljuset             x    x             x               x        0           2   
Macleo             x    x        

## Calcolo dei risultati

### Risultati della terza giornata

In [46]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    pen_1=risultati.loc[girone,partita]["penalità"].iloc[0]
    pen_2=risultati.loc[girone,partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[girone,partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[girone,partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[girone,partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[girone,partita]["match 2 goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["girone","partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["girone","partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano  \
girone partita squadra                                         
A      1       La threesome                           mina99   
               Gli ultimi                           crazycat   
B      1       Green House                       Bango Skank   
               the 3 BrOthers                     AryaSophia   
       2       I Bananari                     simone volponi   
               Gli scrittori del quartierino         Marty12   
C      1       Living Stones                          Macleo   
               Armata Brancaleone                    Ella F.   
       2       Retroguardia Trash                   Andrea28   
               Le TreMende                               Emy   

                                                       riserva  \
girone partita squadra                                           
A      1       La threesome                           Komorebi   
               Gli ultimi                                Steto   
B      1       Green House                              Joyopi   
               the 3 BrOthers                       AryaSophia   
       2       I Bananari                                 Niko   
               Gli scrittori del quartierino           Marty12   
C      1       Living Stones                  Vincenzo Iennaco   
               Armata Brancaleone                     Skorpiòs   
       2       Retroguardia Trash                     Andrea28   
               Le TreMende                                 Emy   

                                                       match 1  \
girone partita squadra                                           
A      1       La threesome                               Luna   
               Gli ultimi                             crazycat   
B      1       Green House                         Bango Skank   
               the 3 BrOthers                 Patrick D'anello   
       2       I Bananari                           massimopud   
               Gli scrittori del quartierino         camparino   
C      1       Living Stones                            Ljuset   
               Armata Brancaleone                      Ella F.   
       2       Retroguardia Trash                        Eudes   
               Le TreMende                              sefora   

                                                     match 2  match 1 goal  \
girone partita squadra                                                       
A      1       La threesome                           mina99             0   
               Gli ultimi                         caipiroska            15   
B      1       Green House                             Plata            11   
               the 3 BrOthers                       ettore70             3   
       2       I Bananari                     simone volponi             6   
               Gli scrittori del quartierino      Federico72             8   
C      1       Living Stones                          Macleo            12   
               Armata Brancaleone                      AdStr             2   
       2       Retroguardia Trash                  AlexComan             9   
               Le TreMende                              Thea             5   

                                              match 1 riserva  match 2 goal  \
girone partita squadra                                                        
A      1       La threesome                             False             5   
               Gli ultimi                               False            11   
B      1       Green House                              False            14   
               the 3 BrOthers                           False             0   
       2       I Bananari                               False             8   
               Gli scrittori del quartierino            False             6   
C      1       Living Stones                            False            10   
               Armata Brancaleone                

In [47]:
schedina

,girone,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,A,1,La threesome,Gli ultimi,5,26,0,15,5,11,-1,0
1,B,1,Green House,the 3 BrOthers,25,3,11,3,14,0,0,0
2,B,2,I Bananari,Gli scrittori del quartierino,14,14,6,8,8,6,0,0
3,C,1,Living Stones,Armata Brancaleone,22,6,12,2,10,4,0,0
4,C,2,Retroguardia Trash,Le TreMende,11,17,9,5,2,12,0,0


In [48]:
out_schedina

titolo match 1 match 2  \
girone partita                                                               
A      1                         La threesome - Gli ultimi  0 - 15  5 - 11   
B      1                      Green House - the 3 BrOthers  11 - 3  14 - 0   
       2        I Bananari - Gli scrittori del quartierino   6 - 8   8 - 6   
C      1                Living Stones - Armata Brancaleone  12 - 2  10 - 4   
       2                  Retroguardia Trash - Le TreMende   9 - 5  2 - 12   

               risultato  
girone partita            
A      1          5 - 26  
B      1          25 - 3  
       2         14 - 14  
C      1          22 - 6  
       2         11 - 17

In [49]:
# output per il post dei risultati

mu.to_table_html_multi(out_schedina)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>La threesome - Gli ultimi</td>
<td style='text-align:center;'>0 - 15</td>
<td style='text-align:center;'>5 - 11</td>
<td style='text-align:center;'>5 - 26</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House - the 3 BrOthers</td>
<td style='text-align:center;'>11 - 3</td>
<td style='text-align:center;'>14 - 0</td>
<td style='text-align:center;'>25 - 3</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari - Gli scrittori del quartierino</td>
<td style='text-align:center;'>6 - 8</td>
<td style='text-align:center;'>8 - 6</td>
<td style='text-align:center;'>14 - 14</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones - Armata Brancaleone</td>
<

### Classifica


Ordine dei criteri:
* classifiche avulse (scontri diretti);
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti nell'ultimo incontro;
* numero di reti segnate nell'ultimo incontro;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.

Nella prima giornata non ci sono classifiche avulse e l'unico incontro è anche l'ultimo, quindi si considerano solo:
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.


In [50]:
## Tabella degli scontri diretti

sg1=schedina_g1.copy()
sg2=schedina_g2.copy()
sg3=schedina.copy()

diretti=sg1[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]].copy()
diretti=diretti.append(sg2[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]])
diretti=diretti.append(sg3[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]])
d=diretti.copy()
diretti.columns=["girone","squadra","avversario","fatti","subiti"]
d.columns=["girone","avversario","squadra","subiti","fatti"]

diretti=diretti.append(d).set_index(["girone","squadra","avversario"])[["fatti","subiti"]].sort_index()
diretti

fatti  \
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                        24   
                                     La threesome                      21   
       Gli ultimi                    D Euforiche                        8   
                                     La threesome                      26   
       La threesome                  D Euforiche                       11   
                                     Gli ultimi                         5   
B      Gli scrittori del quartierino Green House                        7   
                                     I Bananari                        14   
                                     the 3 BrOthers                    16   
       Green House                   Gli scrittori del quartierino     21   
                                     I Bananari                        12   
                                     the 3 BrOthers                    25   
       I Bananari                    Gli scrittori del quartierino     14   
                                     Green House                       16   
                                     the 3 BrOthers                    15   
       the 3 BrOthers                Gli scrittori del quartierino     12   
                                     Green House                        3   
                                     I Bananari                        13   
C      Armata Brancaleone            Le TreMende                       18   
                                     Living Stones                      6   
                                     Retroguardia Trash                17   
       Le TreMende                   Armata Brancaleone                10   
                                     Living Stones                     14   
                                     Retroguardia Trash                17   
       Living Stones                 Armata Brancaleone                22   
                                     Le TreMende                       14   
                                     Retroguardia Trash                 7   
       Retroguardia Trash            Armata Brancaleone                10   
                                     Le TreMende                       11   
                                     Living Stones                     21   

                                                                    subiti  
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                          8  
                                     La threesome                       11  
       Gli ultimi                    D Euforiche                        24  
                                     La threesome                        5  
       La threesome                  D Euforiche                        21  
                                     Gli ultimi                         26  
B      Gli scrittori del quartierino Green House                        21  
                                     I Bananari                         14  
                                     the 3 BrOthers                     12  
       Green House                   Gli scrittori del quartierino       7  
                                     I Bananari                         16  
                                     the 3 BrOthers                      3  
       I Bananari                    Gli scrittori del quartierino      14  
                                     Green House                        12  
                                     the 3 BrOthers                     13  
       the 3 BrOthers                Gli scrittori del quartierino      16  
                                     Green House                        25  
                                     I Bananari                         15  
C      Armata Brancaleone            Le TreMende               

In [51]:
def calc_vittorie(row):
    row["vittorie"]=0
    row["sconfitte"]=0
    row["pareggi"]=0
    if row["goal fatti"]>row["goal subiti"]:
        row["vittorie"]=1
    elif row["goal fatti"]<row["goal subiti"]:
        row["sconfitte"]=1
    else:
        row["pareggi"]=1
    #row["punti"]=3*row["vittorie"]+row["pareggi"]
    return row

q=schedina.copy()

a=q[["girone","squadra 1","goal squadra 1","goal squadra 2",
     "penalità squadra 1","penalità squadra 2"]].copy()
b=q[["girone","squadra 2","goal squadra 2","goal squadra 1",
     "penalità squadra 2","penalità squadra 1"]].copy()

a.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]
b.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]

classifica_g3=a.append(b).set_index(["girone","squadra"]).sort_index()
classifica_g3=classifica_g3.apply(calc_vittorie,axis=1)

df=pd.DataFrame([["A",riposano.iloc[0]["squadra 1"],0]],
                columns=["girone","squadra","goal fatti"])
df=df.set_index(["girone","squadra"])
classifica_g3=classifica_g3.append(df).fillna(0).sort_index()


classifica_g3["goal fatti ultimo incontro"]=classifica_g3["goal fatti"]
classifica_g3["goal subiti ultimo incontro"]=classifica_g3["goal subiti"]
#classifica_g1["goal fatti ultimo incontro"]=0
#classifica_g1["goal subiti ultimo incontro"]=0
classifica_g2["goal fatti ultimo incontro"]=0
classifica_g2["goal subiti ultimo incontro"]=0

base_fields=["goal fatti","goal subiti","penalità fatte",
             "penalità subite","vittorie","sconfitte","pareggi",
             "goal fatti ultimo incontro","goal subiti ultimo incontro"]

#classifica_g3.merge(classifica_g1[base_fields],left_index=True,right_index=True,
#                    how="outer",suffixes=[" g1"," g3"]).fillna(0)
#classifica_g3.merge(classifica_g2[base_fields],left_index=True,right_index=True,
#                    how="outer",suffixes=[" g2"," g3"]).fillna(0)

classifica=classifica_g3+classifica_g2[base_fields]

classifica["goal netti fatti"]=classifica["goal fatti"]-classifica["penalità fatte"]
classifica["goal netti subiti"]=classifica["goal subiti"]-classifica["penalità subite"]

classifica["differenza reti"]=classifica["goal fatti"]-classifica["goal subiti"]
classifica["differenza reti netta"]=classifica["goal netti fatti"]-classifica["goal netti subiti"]
classifica["differenza reti ultimo incontro"]=classifica["goal fatti ultimo incontro"]-classifica["goal subiti ultimo incontro"]

classifica["punti"]=3*classifica["vittorie"]+classifica["pareggi"]

classifica=classifica[["punti","vittorie","pareggi","sconfitte",
                       "differenza reti",
                       "goal fatti","goal subiti",
                       "differenza reti netta",
                       "goal netti fatti","goal netti subiti",
                       "penalità fatte","penalità subite",
                       "differenza reti ultimo incontro",
                       "goal fatti ultimo incontro","goal subiti ultimo incontro"]]
classifica=classifica.astype(int)

In [52]:
importlib.reload(mu)

classifica=mu.sort_classifica(classifica.reset_index(),diretti)
classifica=classifica.set_index(["girone","squadra"])
classifica



punti  vittorie  pareggi  sconfitte  \
girone squadra                                                              
A      D Euforiche                        6         2        0          0   
       Gli ultimi                         3         1        0          1   
       La threesome                       0         0        0          2   
B      I Bananari                         7         2        1          0   
       Green House                        6         2        0          1   
       Gli scrittori del quartierino      4         1        1          1   
       the 3 BrOthers                     0         0        0          3   
C      Armata Brancaleone                 6         2        0          1   
       Living Stones                      4         1        1          1   
       Le TreMende                        4         1        1          1   
       Retroguardia Trash                 3         1        0          2   

                                      differenza reti  goal fatti  \
girone squadra                                                      
A      D Euforiche                                 26          45   
       Gli ultimi                                   5          34   
       La threesome                               -31          16   
B      I Bananari                                   6          45   
       Green House                                 32          58   
       Gli scrittori del quartierino              -10          37   
       the 3 BrOthers                             -28          28   
C      Armata Brancaleone                          -1          41   
       Living Stones                                2          43   
       Le TreMende                                 -2          41   
       Retroguardia Trash                           1          42   

                                      goal subiti  differenza reti netta  \
girone squadra                                                             
A      D Euforiche                             19                     26   
       Gli ultimi                              29                      4   
       La threesome                            47                    -30   
B      I Bananari                              39                      6   
       Green House                             26                     32   
       Gli scrittori del quartierino           47                    -10   
       the 3 BrOthers                          56                    -28   
C      Armata Brancaleone                      42                     -2   
       Living Stones                           41                      2   
       Le TreMende                             43                     -2   
       Retroguardia Trash                      41                      2   

                                      goal netti fatti  goal netti subiti  \
girone squadra                                                              
A      D Euforiche                                  45                 19   
       Gli ultimi                                   34                 30   
       La threesome                                 17                 47   
B      I Bananari                                   45                 39   
       Green House                                  58                 26   
       Gli scrittori del quartierino                37                 47   
       the 3 BrOthers                               28                 56   
C      Armata Brancaleone                           41                 43   
       Living Stones                                43                 41   
       Le TreMende                                  41                 43   
       Retroguardia Trash                           43                 41   

                                      penalità fatte  penalità subite  \
girone squadra                                                          
A

In [53]:
diretti

fatti  \
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                        24   
                                     La threesome                      21   
       Gli ultimi                    D Euforiche                        8   
                                     La threesome                      26   
       La threesome                  D Euforiche                       11   
                                     Gli ultimi                         5   
B      Gli scrittori del quartierino Green House                        7   
                                     I Bananari                        14   
                                     the 3 BrOthers                    16   
       Green House                   Gli scrittori del quartierino     21   
                                     I Bananari                        12   
                                     the 3 BrOthers                    25   
       I Bananari                    Gli scrittori del quartierino     14   
                                     Green House                       16   
                                     the 3 BrOthers                    15   
       the 3 BrOthers                Gli scrittori del quartierino     12   
                                     Green House                        3   
                                     I Bananari                        13   
C      Armata Brancaleone            Le TreMende                       18   
                                     Living Stones                      6   
                                     Retroguardia Trash                17   
       Le TreMende                   Armata Brancaleone                10   
                                     Living Stones                     14   
                                     Retroguardia Trash                17   
       Living Stones                 Armata Brancaleone                22   
                                     Le TreMende                       14   
                                     Retroguardia Trash                 7   
       Retroguardia Trash            Armata Brancaleone                10   
                                     Le TreMende                       11   
                                     Living Stones                     21   

                                                                    subiti  
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                          8  
                                     La threesome                       11  
       Gli ultimi                    D Euforiche                        24  
                                     La threesome                        5  
       La threesome                  D Euforiche                        21  
                                     Gli ultimi                         26  
B      Gli scrittori del quartierino Green House                        21  
                                     I Bananari                         14  
                                     the 3 BrOthers                     12  
       Green House                   Gli scrittori del quartierino       7  
                                     I Bananari                         16  
                                     the 3 BrOthers                      3  
       I Bananari                    Gli scrittori del quartierino      14  
                                     Green House                        12  
                                     the 3 BrOthers                     13  
       the 3 BrOthers                Gli scrittori del quartierino      16  
                                     Green House                        25  
                                     I Bananari                         15  
C      Armata Brancaleone            Le TreMende               

In [54]:
## Output per il post della classifica

mu.to_table_html_multi(classifica)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>punti</th>
<th>vittorie</th>
<th>pareggi</th>
<th>sconfitte</th>
<th>differenza reti</th>
<th>goal fatti</th>
<th>goal subiti</th>
<th>differenza reti netta</th>
<th>goal netti fatti</th>
<th>goal netti subiti</th>
<th>penalità fatte</th>
<th>penalità subite</th>
<th>differenza reti ultimo incontro</th>
<th>goal fatti ultimo incontro</th>
<th>goal subiti ultimo incontro</th>
</tr>
<tr>
<th rowspan='3'>A</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>6</td>
<td style='text-align:center;'>2</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>26</td>
<td style='text-align:center;'>45</td>
<td style='text-align:center;'>19</td>
<td style='text-align:center;'>26</td>
<td style='text-align:center;'>45</td>
<td style='text-align:center;'>19</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td 

In [55]:
# diretti
mu.to_table_html_multi(diretti)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>avversario</th>
<th>fatti</th>
<th>subiti</th>
</tr>
<tr>
<th rowspan='6'>A</th>
<th rowspan='2'>D Euforiche</th>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>24</td>
<td style='text-align:center;'>8</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>21</td>
<td style='text-align:center;'>11</td>
</tr>
<tr>
<th rowspan='2'>Gli ultimi</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>8</td>
<td style='text-align:center;'>24</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>26</td>
<td style='text-align:center;'>5</td>
</tr>
<tr>
<th rowspan='2'>La threesome</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>11</td>
<td style='text-align:center;'>21</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>5</td>
<td style='text-align:center;'>26</td>
</tr>
<tr>
<th

## Squadre promosse

Vengono divise in due gruppi:
* gruppo 1: le tre prime e la migliore seconda;
* gruppo 2: le altre seconde e le due migliori terze.

La classifica sia per le seconde che per le terze è data da, nell'ordine:
* punti normalizzati;
* differenza reti normalizzata.


In [66]:
classifica["piazzamento"]=[1,2,3,1,2,3,4,1,2,3,4]
prime=classifica.reset_index().set_index(["piazzamento","girone"]).loc[1][["squadra","punti",
                                                                           "differenza reti"]]
seconde=classifica.reset_index().set_index(["piazzamento","girone"]).loc[2][["squadra","punti",
                                                                           "differenza reti"]]
terze=classifica.reset_index().set_index(["piazzamento","girone"]).loc[3][["squadra","punti",
                                                                           "differenza reti"]]
def coefficienti(row):
    if row["girone"]=="A":
        row["coefficiente punti"]=3/2
        row["coefficiente goal"]=7/8
    else:
        row["coefficiente punti"]=1
        row["coefficiente goal"]=1
    row["punti normalizzati"]=row["punti"]*row["coefficiente punti"]
    row["differenza reti normalizzata"]=row["differenza reti"]*row["coefficiente goal"]
    return row

seconde=seconde.reset_index().apply(coefficienti,axis=1)
seconde=seconde.sort_values(by=["punti normalizzati","differenza reti normalizzata"],ascending=False)

terze=terze.reset_index().apply(coefficienti,axis=1)
terze=terze.sort_values(by=["punti normalizzati","differenza reti normalizzata"],ascending=False)

terze

gruppo_1=prime.reset_index()[["squadra","girone"]].copy()
gruppo_1=gruppo_1.append(seconde.iloc[:1][["squadra","girone"]])
gruppo_2=seconde.iloc[1:][["squadra","girone"]].copy()
gruppo_2=gruppo_2.append(terze.iloc[:2][["squadra","girone"]])

trovato=False
while not trovato:
    sel_1=[0,1,2,3]
    sel_2=[0,1,2,3]
    random.shuffle(sel_1)
    random.shuffle(sel_2)
    trovato=True
    data=[]
    for n in range(4):
        ind_1=sel_1[n]
        ind_2=sel_2[n]
        if gruppo_1.iloc[ind_1]["girone"]==gruppo_2.iloc[ind_2]["girone"]:
            trovato=False
            break
        data.append( (gruppo_1.iloc[ind_1]["squadra"],
                      gruppo_2.iloc[ind_2]["squadra"]) )
      

## componente casuale da rimuovere
data=[['D Euforiche', 'Gli scrittori del quartierino'],
       ['I Bananari', 'Le TreMende'],
       ['Green House', 'Living Stones'],
       ['Armata Brancaleone', 'Gli ultimi']]
    
quarti=pd.DataFrame(data,columns=["squadra 1","squadra 2"])
quarti

,squadra 1,squadra 2
0,D Euforiche,Gli scrittori del quartierino
1,I Bananari,Le TreMende
2,Green House,Living Stones
3,Armata Brancaleone,Gli ultimi


In [71]:
#mu.to_table_html(quarti)

finali=quarti.copy()
finali["quarti di finale"]=finali["squadra 1"]+" - "+finali["squadra 2"]
finali["semifinali"]=""
finali["finali"]=""
mu.to_table_html(finali[["quarti di finale","semifinali","finali"]])

<table style='margin:auto;' border='1' width='90%'>
<th>quarti di finale</th>
<th>semifinali</th>
<th>finali</th>
</tr>
<tr>
<td style='text-align:center;'>D Euforiche - Gli scrittori del quartierino</td>
<td style='text-align:center;'></td>
<td style='text-align:center;'></td>
</tr>
<tr>
<td style='text-align:center;'>I Bananari - Le TreMende</td>
<td style='text-align:center;'></td>
<td style='text-align:center;'></td>
</tr>
<tr>
<td style='text-align:center;'>Green House - Living Stones</td>
<td style='text-align:center;'></td>
<td style='text-align:center;'></td>
</tr>
<tr>
<td style='text-align:center;'>Armata Brancaleone - Gli ultimi</td>
<td style='text-align:center;'></td>
<td style='text-align:center;'></td>
</tr>
</table>


In [77]:
quarti["partita"]=["Q1","Q2","Q3","Q4"]
quarti["vs"]="contro"
quarti=quarti.set_index("partita")
mu.to_table_html(quarti[["squadra 1","vs","squadra 2"]],with_index=True)


<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th>Q1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
</tr>
<tr>
<th>Q2</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Le TreMende</td>
</tr>
<tr>
<th>Q3</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Living Stones</td>
</tr>
<tr>
<th>Q4</th>
<td style='text-align:center;'>Armata Brancaleone</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli ultimi</td>
</tr>
</table>


### Nuovi punteggi Elo

In [57]:
q=risultati.copy()

def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["girone","partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["girone","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()
#elo["elo precedente"]=1500
#elo["elo avversario"]=1500

elo_g_prec=elo_g_g2

base_elo=elo_g_prec[["elo"]].copy()
base_elo=base_elo.loc[elo.reset_index()["giocatore"]].fillna(1500)

base_elo.columns=["elo precedente"]
elo=elo.reset_index().set_index("giocatore").merge(base_elo,left_index=True,right_index=True)
base_elo.columns=["elo avversario"]
elo=elo.reset_index().set_index("avversario").merge(base_elo,left_index=True,right_index=True)
elo=elo.reset_index().set_index(["girone","squadra","giocatore"])
elo.columns=["avversario","goal fatti","goal subiti","elo precedente","elo avversario"]
elo=elo.apply(calcola_punteggi,axis=1)
elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]
elo=elo.reset_index().set_index(["giocatore"])
elo


,girone,squadra,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento
giocatore,,,,,,,,,,
caipiroska,A,Gli ultimi,mina99,11,5,1468.0,1500.0,0.454078,0.687500,0.233422
crazycat,A,Gli ultimi,Luna,15,0,1490.0,1500.0,0.485613,1.000000,0.514387
Luna,A,La threesome,crazycat,0,15,1500.0,1490.0,0.514387,0.000000,-0.514387
mina99,A,La threesome,caipiroska,5,11,1500.0,1468.0,0.545922,0.312500,-0.233422
Federico72,B,Gli scrittori del quartierino,simone volponi,6,8,1493.0,1486.0,0.510072,0.428571,-0.081501
camparino,B,Gli scrittori del quartierino,massimopud,8,6,1494.0,1501.0,0.489928,0.571429,0.081501
Bango Skank,B,Green House,Patrick D'anello,11,3,1507.0,1500.0,0.510072,0.785714,0.275642
Plata,B,Green House,ettore70,14,0,1506.0,1499.0,0.510072,1.000000,0.489928
massimopud,B,I Bananari,camparino,6,8,1501.0,1494.0,0.510072,0.428571,-0.081501


In [58]:
elo_s_prec=elo_g_prec.reset_index().set_index("squadra").sort_index().groupby(level=0).mean().copy()

a=schedina.copy()[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]]
b=schedina.copy()[["girone","squadra 2","squadra 1","goal squadra 2","goal squadra 1"]]
a.columns=["girone","squadra","avversario","goal fatti","goal subiti"]
b.columns=["girone","squadra","avversario","goal fatti","goal subiti"]

elo_s=a.append(b).set_index("squadra")

elo_s_prec.columns=["elo precedente"]
elo_s=elo_s.merge(elo_s_prec,left_index=True,right_index=True)
elo_s_prec.columns=["elo avversario"]
elo_s=elo_s.reset_index().set_index("avversario").merge(elo_s_prec,left_index=True,
                                                        right_index=True)

elo_s=elo_s.reset_index().set_index("squadra")
elo_s.columns=["avversario","girone","goal fatti","goal subiti","elo precedente","elo avversario"]
elo_s=elo_s.apply(calcola_punteggi,axis=1)
elo_s["scostamento squadra"]=elo_s["punteggio reale"]-elo_s["punteggio atteso"]

rip_sq=riposano.iloc[0]["squadra 1"]
rip_mean=elo_g_prec[elo_g_prec["squadra"]==rip_sq]["elo"].mean()

rip=pd.DataFrame([[rip_sq,"-","A",0,0,rip_mean,rip_mean,0.5,0.5,0.0]],
                 columns=["squadra"]+list(elo_s.columns))

elo_s=elo_s.append(rip.set_index("squadra"))
elo_s

,avversario,girone,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento squadra
squadra,,,,,,,,,
La threesome,Gli ultimi,A,5,26,1500.000000,1484.000000,0.523010,0.161290,-0.361719
Green House,the 3 BrOthers,B,25,3,1513.000000,1499.000000,0.520137,0.892857,0.372720
I Bananari,Gli scrittori del quartierino,B,14,14,1506.000000,1479.000000,0.538778,0.500000,-0.038778
Living Stones,Armata Brancaleone,C,22,6,1489.000000,1523.333333,0.450751,0.785714,0.334964
Retroguardia Trash,Le TreMende,C,11,17,1501.333333,1486.333333,0.521573,0.392857,-0.128716
Gli ultimi,La threesome,A,26,5,1484.000000,1500.000000,0.476990,0.838710,0.361719
the 3 BrOthers,Green House,B,3,25,1499.000000,1513.000000,0.479863,0.107143,-0.372720
Gli scrittori del quartierino,I Bananari,B,14,14,1479.000000,1506.000000,0.461222,0.500000,0.038778
Armata Brancaleone,Living Stones,C,6,22,1523.333333,1489.000000,0.549249,0.214286,-0.334964


In [59]:

q=elo_s.reset_index()[["girone","squadra","scostamento squadra"]]
#elo.merge(d,on="squadra")
q=squadre.reset_index().merge(q,on="squadra").set_index(["giocatore","squadra","girone"])
q=elo_g_prec.reset_index().set_index(["squadra",
                                    "giocatore"]).merge(q.reset_index().set_index(["squadra",
                                                                                   "giocatore"]),
                                                        left_index=True,right_index=True,
                                                        how="outer")
q=q.reset_index().set_index(["giocatore","squadra","girone"])

#elo=elo.copy()

elo=elo.reset_index().set_index(["giocatore","squadra","girone"])
elo=elo.merge(q[["scostamento squadra","elo"]],left_index=True,right_index=True,how="outer")
elo["elo precedente"]=elo["elo"]

elo["goal fatti"]=pd.to_numeric(elo["goal fatti"].fillna(0),downcast="signed")
elo["goal subiti"]=pd.to_numeric(elo["goal subiti"].fillna(0),downcast="signed")
elo["elo precedente"]=pd.to_numeric(elo["elo precedente"].fillna(1500),downcast="signed")
elo["elo avversario"]=pd.to_numeric(elo["elo avversario"].fillna(1500),downcast="signed")
elo["punteggio atteso"]=elo["punteggio atteso"].fillna(0.5)
elo["punteggio reale"]=elo["punteggio reale"].fillna(0.5)
elo["scostamento"]=elo["scostamento"].fillna(0.0)
elo["avversario"]=elo["avversario"].fillna("(non giocato)")

elo=elo.reset_index().set_index(["giocatore"])

L=list(elo.columns)
L.remove("elo")
elo=elo[L]


In [62]:
def calc_K(dfcol,max_variazione):
    mean=dfcol.mean()
    sigma=dfcol.std()
    for alpha in np.linspace(0,6,601):
        if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
            break
    max_scostamento=alpha*sigma
    K=max_variazione/max_scostamento
    return K

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)

KG=calc_K(elo["scostamento"],70)
KS=calc_K(elo["scostamento squadra"],20)


elo["elo nuovo"]=elo["elo precedente"]+KG*elo["scostamento"]+KS*elo["scostamento squadra"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)

elo[["elo precedente","elo nuovo","scostamento","scostamento squadra"]].sort_values("elo nuovo",ascending=False)

,elo precedente,elo nuovo,scostamento,scostamento squadra
giocatore,,,,
Ella F.,1567,1562,0.000000,-0.334964
Kuno,1557,1557,0.000000,0.000000
Plata,1506,1544,0.489928,0.372720
ITG,1548,1542,0.000000,-0.372720
Joyopi,1526,1532,0.000000,0.372720
Bango Skank,1507,1531,0.275642,0.372720
Niko,1531,1530,0.000000,-0.038778
crazycat,1490,1530,0.514387,0.361719
Ljuset,1498,1527,0.365777,0.334964


In [63]:
a=gironi[gironi["A"]!="(riposo)"][["A"]].copy()
a.columns=["squadra"]
a["girone"]="A"
a

b=gironi[gironi["B"]!="(riposo)"][["B"]].copy()
b.columns=["squadra"]
b["girone"]="B"
b

c=gironi[gironi["C"]!="(riposo)"][["C"]].copy()
c.columns=["squadra"]
c["girone"]="C"
c

q=a.append(b).append(c).set_index("squadra")
q=q.merge(squadre,left_index=True,right_index=True,how="outer")
q=q.reset_index().set_index("giocatore")
q

elo_g=elo.merge(q,how="outer",
                left_index=True,right_index=True)
elo_g=elo_g[["squadra_y","elo nuovo"]].fillna(1500).sort_values("elo nuovo",
                                                                ascending=False)
elo_g["elo nuovo"]=pd.to_numeric(elo_g["elo nuovo"],downcast="signed")
elo_g.columns=["squadra","elo"]
elo_g


,squadra,elo
giocatore,,
Ella F.,Armata Brancaleone,1562
Kuno,D Euforiche,1557
Plata,Green House,1544
ITG,the 3 BrOthers,1542
Joyopi,Green House,1532
Bango Skank,Green House,1531
Niko,I Bananari,1530
crazycat,Gli ultimi,1530
Ljuset,Living Stones,1527


In [64]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>Ella F.</th>
<td style='text-align:center;'>Armata Brancaleone</td>
<td style='text-align:center;'>1562</td>
</tr>
<tr>
<th>Kuno</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1557</td>
</tr>
<tr>
<th>Plata</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1544</td>
</tr>
<tr>
<th>ITG</th>
<td style='text-align:center;'>the 3 BrOthers</td>
<td style='text-align:center;'>1542</td>
</tr>
<tr>
<th>Joyopi</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1532</td>
</tr>
<tr>
<th>Bango Skank</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1531</td>
</tr>
<tr>
<th>Niko</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>1530</td>
</tr>
<tr>
<th>crazycat</th>
<td style='text-align:center;'>Gli ultimi</td>
<td

## Salvataggio dei risultati

In [80]:
## CSV
label="giornata_%d" % GIORNATA

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# classifica
classifica_csv="data/classifica_"+label+".csv"
classifica.to_csv(classifica_csv)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

# elo_g
elo_s_csv="data/elo_s_"+label+".csv"
elo_s.to_csv(elo_s_csv)

# ammissioni
ammissioni_csv="data/ammissioni_"+label+".csv"
ammissioni.to_csv(ammissioni_csv)

# quarti
quarti_csv="data/quarti_"+label+".csv"
quarti.to_csv(quarti_csv)


In [79]:
quarti

,squadra 1,squadra 2,vs
partita,,,
Q1,D Euforiche,Gli scrittori del quartierino,contro
Q2,I Bananari,Le TreMende,contro
Q3,Green House,Living Stones,contro
Q4,Armata Brancaleone,Gli ultimi,contro
